In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ply

## load data and preprocess

save what data in what type file

In [2]:
%%time
questions = pd.read_csv('./question_train_word.csv')
questions_topics = questions.topics.apply(lambda s: s.split(','))
question_titles = questions.title.astype('U').apply(lambda s: s.split(','))

CPU times: user 30.8 s, sys: 16 s, total: 46.8 s
Wall time: 58.8 s


In [3]:
questions.head(1)

,Unnamed: 0,Unnamed: 0.1,id,title,description,topics
0,0,0,6555699376639805223,"w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...","w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...","7739004195693774975,3738968195649774859"


In [4]:
%%time
from collections import Counter

question_words = [w for ws in question_titles for w in ws]
word_counter = Counter(question_words)
word_most_common = word_counter.most_common()

CPU times: user 10.8 s, sys: 5.52 s, total: 16.3 s
Wall time: 19.7 s


#### Using `np.array` could be more efficient then `list`. 

In [5]:
questions_topics = np.array(questions_topics)
questions_title = np.array(question_titles)
print(questions_topics[0])
print(questions_title[0])

['7739004195693774975', '3738968195649774859']
['w305', 'w13549', 'w22752', 'w11', 'w7225', 'w2565', 'w1106', 'w16', 'w31389', 'w6', 'w1019', 'w69288', 'w111', 'w3332', 'w109', 'w11', 'w25', 'w1110', 'w111']


In [6]:
%%time
from collections import Counter
topic_count = Counter([t for ts in questions_topics for t in ts])
print(topic_count.most_common(3))

[('7476760589625268543', 65989), ('4697014490911193675', 49192), ('-4653836020042332281', 45444)]
CPU times: user 1.66 s, sys: 944 ms, total: 2.6 s
Wall time: 3.97 s


In [7]:
topic_most_common = np.array(topic_count.most_common())
print(topic_most_common[0])

['7476760589625268543' '65989']


### 1999 Classifiers y

In [8]:
def tag_index(i, length=len(questions_topics)):
    tag = topic_most_common[i][0]
    y_index = []
    for topics in questions_topics[:int(length)]:
        if tag in topics:
            y_index.append(1)
        else:
            y_index.append(0)
    return y_index

In [9]:
%time y_most_common = tag_index(0)

CPU times: user 665 ms, sys: 115 ms, total: 780 ms
Wall time: 1.02 s


In [35]:
tag_lens = [len(i) for i in questions_topics]
tmp = np.concatenate(questions_topics)
def in_eachlist(l, search_num):
    mask = tmp == search_num
    return np.logical_or.reduceat(mask,np.concatenate(([0], np.cumsum(tag_lens[:-1]))) )

In [39]:
%time y_most_common_1 = in_eachlist(questions_topics, topic_most_common[0][0])

CPU times: user 388 ms, sys: 25.2 ms, total: 413 ms
Wall time: 414 ms


In [33]:
print(y_most_common[:10])
print(y_most_common_1[:10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[False False False False False False False False False False]


## stop words

In [11]:
word_low_frequency = [w[0] for w in word_most_common if w[1] <= 5]

In [14]:
print(len(word_most_common))
print(len(word_low_frequency))

324960
204962


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

## TfidfVectorizer params

- `max_df`
- `min_df`
- `ngram_range`

#### Factors in consideration.

- accuracy, using `gridsearch` to optimize
- time consuming
    - less words, faster speed
    - n-gram might have **higher order of magnitude**

## GridSearchCV

- time consume
- accuracy
    - can the result of one binary classifier represents the accuracy of 1999 classifiers?

In [ ]:
%%time
# check tf-idf vect speed

vect = TfidfVectorizer(max_df=0.85, min_df=5, ngram_range=(1,2))
X_train = vect.fit_transform(questions.title.astype('U'))

In [12]:
X_train.shape

(2999967, 894346)

In [60]:
vect_v2 = TfidfVectorizer(max_df=0.85, min_df=20, ngram_range=(1,2))
%time X_train_v2 = vect_v2.fit_transform(questions.title.astype('U'))
X_train_v2.shape

CPU times: user 3min 13s, sys: 10.8 s, total: 3min 24s
Wall time: 3min 35s


(2999967, 232863)

In [13]:
from sklearn.linear_model import SGDClassifier

In [43]:
model = SGDClassifier(alpha=10**-5, loss='modified_huber', n_jobs=-1)
%time model.fit(X_train[:10**4], y_most_common[:10**4])

CPU times: user 38.7 ms, sys: 6.6 ms, total: 45.3 ms
Wall time: 47.1 ms


SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [44]:
model = SGDClassifier(alpha=10**-5, loss='modified_huber', n_jobs=-1, n_iter=1)
%time model.fit(X_train[:10**4], y_most_common[:10**4])

CPU times: user 14.2 ms, sys: 4.88 ms, total: 19 ms
Wall time: 17 ms


SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=1, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [45]:
model = SGDClassifier(alpha=10**-5, loss='modified_huber', n_jobs=-1, n_iter=1)
%time model.fit(X_train, y_most_common)

CPU times: user 2.53 s, sys: 1.54 s, total: 4.08 s
Wall time: 5.6 s


SGDClassifier(alpha=1e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=1, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

## Optimize SGD params

### Choosing the reasonable `scoring` is important

In [46]:
from sklearn.model_selection import GridSearchCV

In [77]:
params = {'alpha': 10**np.arange(-10,-5, dtype=float), 'n_iter': [1,3,5]}
grid = GridSearchCV(SGDClassifier(loss='modified_huber', n_jobs=-1), param_grid=params, n_jobs=-1, scoring='recall')

In [81]:
%time grid.fit(X_train_v2[:10**5], y_most_common[:10**5])
print(grid.best_params_)
grid.cv_results_

CPU times: user 7.1 s, sys: 22.3 s, total: 29.4 s
Wall time: 47.8 s
{'alpha': 9.9999999999999995e-08, 'n_iter': 1}


{'mean_fit_time': array([ 0.44885969,  0.52336065,  0.61670129,  0.35725236,  0.40181645,
         0.48096697,  0.2586472 ,  0.31735714,  0.49081119,  0.28111172,
         0.42776863,  0.46219897,  0.25623949,  0.33877182,  0.33589641]),
 'mean_score_time': array([ 0.05107538,  0.04293402,  0.05891426,  0.05032269,  0.03461909,
         0.03070807,  0.0314587 ,  0.03524955,  0.03147046,  0.05263591,
         0.02711741,  0.03168575,  0.03888583,  0.0299499 ,  0.01723433]),
 'mean_test_score': array([ 0.05975525,  0.03925213,  0.0348875 ,  0.07980201,  0.03881194,
         0.03706174,  0.08240697,  0.05277452,  0.04274083,  0.08678442,
         0.04622562,  0.03008992,  0.0675835 ,  0.03838233,  0.02965976]),
 'mean_train_score': array([ 0.7612211 ,  0.90732402,  0.95159008,  0.8246888 ,  0.91016268,
         0.95988221,  0.79896392,  0.91910241,  0.96380178,  0.79350132,
         0.92672854,  0.96075025,  0.79743459,  0.94046885,  0.9866977 ]),
 'param_alpha': masked_array(data = [1e-1

In [78]:
%time grid.fit(X_train[:10**5], y_most_common[:10**5])

CPU times: user 7.18 s, sys: 22.1 s, total: 29.3 s
Wall time: 44.8 s


GridSearchCV(cv=None, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'alpha': array([  1.00000e-10,   1.00000e-09,   1.00000e-08,   1.00000e-07,
         1.00000e-06]), 'n_iter': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=0)

In [79]:
print(grid.best_params_)
grid.cv_results_

{'alpha': 9.9999999999999995e-08, 'n_iter': 1}


{'mean_fit_time': array([ 0.55363361,  0.51156791,  0.49278593,  0.30294998,  0.4300673 ,
         0.51456364,  0.31081223,  0.38770509,  0.49239516,  0.31108093,
         0.38887612,  0.46929765,  0.3062652 ,  0.41853786,  0.41771897]),
 'mean_score_time': array([ 0.03201501,  0.03833834,  0.03407439,  0.03110655,  0.06620797,
         0.03215432,  0.03000808,  0.03049628,  0.03039336,  0.02881972,
         0.03358698,  0.03683917,  0.03507447,  0.0280629 ,  0.02095946]),
 'mean_test_score': array([ 0.06977612,  0.03053346,  0.02180755,  0.06628353,  0.01831439,
         0.01744626,  0.10205533,  0.03009159,  0.02311643,  0.11076118,
         0.02573142,  0.0191881 ,  0.08897258,  0.02572919,  0.01788088]),
 'mean_train_score': array([ 0.86829527,  0.97513988,  0.99062511,  0.88334292,  0.96249673,
         0.97361269,  0.91059513,  0.96511154,  0.98059078,  0.91561315,
         0.96751005,  0.98408189,  0.91517471,  0.9810295 ,  0.99869153]),
 'param_alpha': masked_array(data = [1e-1

In [14]:
from tqdm import tqdm_notebook

def predict(clf_list, length=2999967, start=0, word_tfidf=X_train):
    predict_list = []
    for clf in tqdm_notebook(clf_list):
        try:
            result = [1-j[0] for j in clf.predict_proba(word_tfidf[start:start+length])]
        except:
            result = np.zeros(length)
        predict_list.append(result)
    return predict_list

In [87]:
sgd_list = [SGDClassifier(loss="modified_huber", penalty="l2", n_jobs=-1, alpha=1e-7, n_iter=1) for _ in range(1999)]

for i in tqdm_notebook(range(1999)):
    sgd = sgd_list[i]
    y_tag = tag_index(i, 10**5)
    sgd.fit(X_train[:10**5], y_tag[:10**5]) 
#     joblib.dump(clf, 'sgd_classifier_v2/sgd_{}.pkl'.format(i)) 

In [89]:
predict_list = predict(sgd_list, 10**5)

In [15]:
%time topic_map = {i: v[0] for i, v in enumerate(topic_most_common)}

CPU times: user 3.19 ms, sys: 1.73 ms, total: 4.92 ms
Wall time: 12.9 ms


In [43]:
# def _transform(predict_list):
#     result = []
#     for i in range(len(predict_list[0])):
#         _list = []
#         for j in range(len(predict_list)):
#             _list.append(predict_list[j][i])
#         result.append(_list)
#     return result

def transform(predict_list):
    return np.array(predict_list).T

# import heapq
def top_five(predict_list):
    return np.array(predict_list).argsort()[-5:][::-1]
#     return [i[1] for i in heapq.nlargest(5, ((v, i) for i, v in enumerate(predict_list)))]


def transform_predict(predict_list):
    predict_list_t = transform(predict_list)
    result = []
    for p in predict_list_t:
        result.append(list(map(lambda x: topic_map[x], top_five(p))))
    return result

In [91]:
result = transform_predict(predict_list)

In [95]:
from evaluate import evaluate
evaluate(zip(result, questions_topics[:10**5]))

(0.6105787508041599, 2.0676889633119226, 0.8664320196464315, 206041, 237804)

## multiprocessing

In [17]:
import multiprocessing

In [21]:
from sklearn.externals import joblib

def fit(args):
    i, clf = args
    y = tag_index(i)
    clf.fit(X_train, y)
    joblib.dump(clf, 'sgd_classifier_v3/sgd_{}.pkl'.format(i)) 
    return i

In [22]:
sgd_train_list = [(i, SGDClassifier(loss="modified_huber", penalty="l2", n_jobs=-1, alpha=1e-7, n_iter=1)) for i in range(1999)]
sgd_train_list[0]

(0, SGDClassifier(alpha=1e-07, average=False, class_weight=None, epsilon=0.1,
        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
        learning_rate='optimal', loss='modified_huber', n_iter=1, n_jobs=-1,
        penalty='l2', power_t=0.5, random_state=None, shuffle=True,
        verbose=0, warm_start=False))

In [23]:
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
print(cores)

4


In [ ]:
%%time
l = pool.map(fit, sgd_train_list)

CPU times: user 4.44 s, sys: 1.39 s, total: 5.84 s
Wall time: 56min 26s


In [2]:
%%time

from sklearn.externals import joblib

sgd_list = []
for i in range(1999):
    sgd_list.append(joblib.load('sgd_classifier_v3/sgd_{}.pkl'.format(i)))

CPU times: user 7.99 s, sys: 13.4 s, total: 21.4 s
Wall time: 33.9 s


## format test data 

In [ ]:
%%time
test_questions = pd.read_csv('./question_eval_set.txt', sep='\t', names=['id', 'c', 'w', 'd_c', 'd_w'])
X_test = vect.transform(test_questions.w.astype('U'))

In [27]:
X_test.shape

(217360, 894346)

## predict

In [ ]:
%%time
predict_list = predict(sgd_list, 217360, word_tfidf=X_test)

In [44]:
%%time
result = transform_predict(predict_list)

CPU times: user 16.1 s, sys: 6.19 s, total: 22.3 s
Wall time: 28.6 s


In [45]:
%time result = [str(test_questions.id[i])+','+','.join(v) for i, v in enumerate(result)]
result[0]

CPU times: user 4.87 s, sys: 132 ms, total: 5 s
Wall time: 5.15 s


'6215603645409872328,1797701934329285597,6756871902441838709,-2667619839677460873,3359253792326581125,8852526557827312102'

In [46]:
%%time


sgd_list_v1 = []
for i in range(1999):
    sgd_list_v1.append(joblib.load('sgd_classifier/sgd_{}.pkl'.format(i)))

CPU times: user 3.13 s, sys: 2.71 s, total: 5.84 s
Wall time: 10.9 s


In [48]:
predict_list = predict(sgd_list_v1, 10**4)

In [49]:
%time result = transform_predict(predict_list)

CPU times: user 519 ms, sys: 120 ms, total: 639 ms
Wall time: 649 ms


In [51]:
from evaluate import evaluate
evaluate(zip(result, questions_topics[:10**4]))

(0.0008686307301261166, 0.0025033434630589886, 0.001330191547582852, 35, 26312)